In [ ]:
from clique import _CliqueProblem, _CliqueTrial, migrate_old_graph_to_new_system
from clique_benchmark import Trials

from benchmarklib import _BenchmarkDatabase, CompileType

In [ ]:
migrate_old_graph_to_new_system("graphs.db", "clique.db")

In [ ]:
old_db = Trials()
new_db = _BenchmarkDatabase('clique.db', _CliqueProblem, _CliqueTrial)

In [ ]:
with old_db._connect() as conn:
    count = conn.execute("SELECT COUNT(DISTINCT(g)) FROM graphs").fetchone()[0]
    print(f"Number of graphs in old db: {count}")

In [ ]:
with new_db._connect() as conn:
    count = conn.execute("SELECT COUNT(DISTINCT(json_extract(problem_data, '$.graph'))) FROM problem_instances").fetchone()[0]
    print(f"Number of graphs in new db: {count}")

In [ ]:
p = old_db.get()[0]
p.__dict__

In [ ]:
[g.__dict__ for g in new_db.find_problem_instances(problem_data_filters={'graph': str(p.graph)})[:5]]

In [ ]:
for trial in old_db.get():
    problem = new_db.find_problem_instances(
        problem_data_filters={'graph': str(trial.graph)}
    )
    if problem:
        problem = problem[0]
    else:
        print(f"skipping trial for missing graph {trial.graph}")
        continue

    new_trial = _CliqueTrial(
        problem_instance=problem,
        compiler_name=trial.compile_type,
        job_id=trial.job_id,
        job_pub_idx=trial.job_pub_idx,
        counts=trial.counts,
        grover_iterations=trial.grover_iterations,
        clique_size=trial.clique_size,
        simulation_counts=None,
    )
    new_db.save_trial(new_trial)